In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50, EfficientNetB0, resnet50, efficientnet
from tensorflow.keras.optimizers import Adam

# ---------------------------------------------------
# 1. Load the TF Flowers Dataset
# ---------------------------------------------------
# Create train/validation/test split
(dataset_train, dataset_val, dataset_test), info = tfds.load(
    'tf_flowers',
    split=['train[:70%]', 'train[70%:85%]', 'train[85%:]'],
    as_supervised=True,
    with_info=True
)

# Number of classes in TF Flowers
num_classes = info.features['label'].num_classes

# ---------------------------------------------------
# 2. Preprocess Data for ResNet and EfficientNet
# ---------------------------------------------------
IMG_SIZE = 224
batch_size = 32
AUTOTUNE = tf.data.AUTOTUNE

def preprocess_resnet(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = resnet50.preprocess_input(image)
    return image, tf.one_hot(label, num_classes)

def preprocess_efficientnet(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = efficientnet.preprocess_input(image)
    return image, tf.one_hot(label, num_classes)

# Create tf.data pipelines
train_ds_resnet = (dataset_train
                   .map(preprocess_resnet, num_parallel_calls=AUTOTUNE)
                   .batch(batch_size)
                   .prefetch(AUTOTUNE))

val_ds_resnet = (dataset_val
                 .map(preprocess_resnet, num_parallel_calls=AUTOTUNE)
                 .batch(batch_size)
                 .prefetch(AUTOTUNE))

test_ds_resnet = (dataset_test
                  .map(preprocess_resnet, num_parallel_calls=AUTOTUNE)
                  .batch(batch_size)
                  .prefetch(AUTOTUNE))

train_ds_eff = (dataset_train
                .map(preprocess_efficientnet, num_parallel_calls=AUTOTUNE)
                .batch(batch_size)
                .prefetch(AUTOTUNE))

val_ds_eff = (dataset_val
              .map(preprocess_efficientnet, num_parallel_calls=AUTOTUNE)
              .batch(batch_size)
              .prefetch(AUTOTUNE))

test_ds_eff = (dataset_test
               .map(preprocess_efficientnet, num_parallel_calls=AUTOTUNE)
               .batch(batch_size)
               .prefetch(AUTOTUNE))

# ---------------------------------------------------
# 3. Define ResNet50 Model
# ---------------------------------------------------
def create_resnet_model(input_shape, num_classes):
    base_model = ResNet50(weights="imagenet", include_top=False, input_shape=input_shape)
    base_model.trainable = True

    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    model = models.Model(inputs=base_model.input, outputs=outputs, name="ResNet")
    return model

# ---------------------------------------------------
# 4. Define EfficientNetB0 Model
# ---------------------------------------------------
def create_efficientnet_model(input_shape, num_classes):
    base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=input_shape)
    base_model.trainable = True

    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    model = models.Model(inputs=base_model.input, outputs=outputs, name="EfficientNet")
    return model

# ---------------------------------------------------
# 5. Compile and Train the Models
# ---------------------------------------------------
# ResNet50 Model
resnet_model = create_resnet_model((IMG_SIZE, IMG_SIZE, 3), num_classes)
resnet_model.compile(optimizer=Adam(learning_rate=1e-4), loss="categorical_crossentropy", metrics=["accuracy"])

print("\n--- Fine-Tuning ResNet50 on TF Flowers ---")
history_resnet = resnet_model.fit(train_ds_resnet, epochs=3, validation_data=val_ds_resnet, verbose=1)

# EfficientNetB0 Model
eff_model = create_efficientnet_model((IMG_SIZE, IMG_SIZE, 3), num_classes)
eff_model.compile(optimizer=Adam(learning_rate=1e-4), loss="categorical_crossentropy", metrics=["accuracy"])

print("\n--- Fine-Tuning EfficientNetB0 on TF Flowers ---")
history_eff = eff_model.fit(train_ds_eff, epochs=3, validation_data=val_ds_eff, verbose=1)

# Evaluate ResNet50
loss_resnet, acc_resnet = resnet_model.evaluate(test_ds_resnet, verbose=0)
print(f"ResNet50 Final Accuracy: {acc_resnet:.4f}")

# Evaluate EfficientNetB0
loss_eff, acc_eff = eff_model.evaluate(test_ds_eff, verbose=0)
print(f"EfficientNetB0 Final Accuracy: {acc_eff:.4f}")


--- Fine-Tuning ResNet50 on TF Flowers ---
Epoch 1/3
81/81 ━━━━━━━━━━━━━━━━━━━━ 122s 715ms/step - accuracy: 0.6954 - loss: 0.8056 - val_accuracy: 0.8639 - val_loss: 0.5272
Epoch 2/3
81/81 ━━━━━━━━━━━━━━━━━━━━ 26s 312ms/step - accuracy: 0.9814 - loss: 0.0733 - val_accuracy: 0.9274 - val_loss: 0.2106
Epoch 3/3
81/81 ━━━━━━━━━━━━━━━━━━━━ 26s 324ms/step - accuracy: 0.9996 - loss: 0.0093 - val_accuracy: 0.9583 - val_loss: 0.1440

--- Fine-Tuning EfficientNetB0 on TF Flowers ---
Epoch 1/3
81/81 ━━━━━━━━━━━━━━━━━━━━ 134s 742ms/step - accuracy: 0.5341 - loss: 1.1745 - val_accuracy: 0.9002 - val_loss: 0.3349
Epoch 2/3
81/81 ━━━━━━━━━━━━━━━━━━━━ 58s 152ms/step - accuracy: 0.9122 - loss: 0.3212 - val_accuracy: 0.9419 - val_loss: 0.1979
Epoch 3/3
81/81 ━━━━━━━━━━━━━━━━━━━━ 20s 146ms/step - accuracy: 0.9625 - loss: 0.1643 - val_accuracy: 0.9510 - val_loss: 0.1608
ResNet50 Final Accuracy: 0.9491
EfficientNetB0 Final Accuracy: 0.9345
